<a href="https://colab.research.google.com/github/Hamouda-Yasmine/PFE-X-ray_image_search_using_CBIR_deepL/blob/main/CBIR-MAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
print (len(glob.glob('/content/drive/MyDrive/clef2009/train/*.*')))

12941


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
! pip install torch-model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import torch
import torchvision.transforms as T
import os
from sklearn.metrics import average_precision_score, accuracy_score,balanced_accuracy_score
from operator import is_not
from pathlib import Path
import pandas as pd
import csv
import os
import os.path
import cv2
import glob
import matplotlib.pyplot as plt
import shutil
import os
class ConvEncoder(nn.Module):
    """
    A simple Convolutional Encoder Model
    """

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, (3, 3), padding=(1, 1))
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d((2, 2))

        self.conv2 = nn.Conv2d(16, 32, (3, 3), padding=(1, 1))
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d((2, 2))

        self.conv3 = nn.Conv2d(32, 64, (3, 3), padding=(1, 1))
        self.relu3 = nn.ReLU(inplace=True)
        self.maxpool3 = nn.MaxPool2d((2, 2))

        self.conv4 = nn.Conv2d(64, 128, (3, 3), padding=(1, 1))
        self.relu4 = nn.ReLU(inplace=True)
        self.maxpool4 = nn.MaxPool2d((2, 2))

        self.conv5 = nn.Conv2d(128, 256, (3, 3), padding=(1, 1))
        self.relu5 = nn.ReLU(inplace=True)
        self.maxpool5 = nn.MaxPool2d((2, 2))

    def forward(self, x):
        # Downscale the image with conv maxpool etc.
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.maxpool4(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.maxpool5(x)

        return x

In [ ]:
class FolderDataset(Dataset):
    """
    Creates a PyTorch dataset from folder, returning two tensor images.
    Args:
    main_dir : directory where images are stored.
    transform (optional) : torchvision transforms to be applied while making dataset
    """

    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        self.transform = transform
        self.all_imgs = os.listdir(main_dir)

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.all_imgs[idx])
        image = Image.open(img_loc).convert("RGB")

        if self.transform is not None:
            tensor_image = self.transform(image)

        return tensor_image, tensor_image

In [ ]:
full_loader=torch.load('/content/drive/MyDrive/IRMAcbirSGD/fullloaderirma.pth')
print (len(full_loader.dataset.all_imgs))

12941


In [ ]:
import torch
import numpy as np
import torch_model
from sklearn.neighbors import NearestNeighbors
import torchvision.transforms as T
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import cv2


def load_image_tensor(image_path, device):

    # width:   400
# height:  225
# channel: 3
    image_tensor = T.ToTensor()(Image.open(image_path).convert('RGB'))



    image_tensor = image_tensor.unsqueeze(0)
    print(image_tensor.shape)
    # input_images = image_tensor.to(device)
    return image_tensor

In [ ]:
def compute_similar_images(image_path, knn, device):
    image_tensor = load_image_tensor(image_path, device)
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        image_embedding = encoder(image_tensor).cpu().detach().numpy()

    print(image_embedding.shape)

    flattened_embedding = image_embedding.reshape((image_embedding.shape[0], -1))
    print(flattened_embedding.shape)



    _, indices = knn.kneighbors(flattened_embedding)
    indices_list = indices.tolist()
    #print(indices_list)
    return indices_list

In [ ]:

ENCODER_MODEL_PATH = "/content/drive/MyDrive/IRMAcbirSGD/encoder_model_irmaSGD.pt"
EMBEDDING_PATH = "/content/drive/MyDrive/IRMAcbirSGD/data_embedding_SGD-IRMA.npy"

device = torch.device("cpu")
encoder =ConvEncoder()
print (device)
# Load the state dict of encoder
encoder.load_state_dict(torch.load(ENCODER_MODEL_PATH, map_location=device))
encoder.eval()
encoder.to(device)

# Loads the embedding
embedding = np.load(EMBEDDING_PATH)
print(len(embedding))
knn = NearestNeighbors(n_neighbors=10, metric="canberra")
knn.fit(embedding)

cpu
12942


NearestNeighbors(metric='canberra', n_neighbors=10)

In [ ]:
import pandas as pd
csvtrain=pd.read_csv("/content/drive/MyDrive/IRMA/ImageCLEFmed2009_train_codes.02.csv",sep=';')
csvtrain=csvtrain.set_index("image_id",drop=False)
csvtest=pd.read_csv("/content/drive/MyDrive/IRMA/ImageCLEFmed2009_test_codes.03.csv",sep=';')
csvtest=csvtest.set_index("image_id",drop=False)
path1 = "/content/drive/MyDrive/IRMA/train/*.*"

In [ ]:
def Averageprecision( classe,code,indices):
 correctImages = []
 SimImg=0
 compRes=0
 for index in indices:
    pathimagesimilaire=full_loader.dataset.all_imgs[index-1]
    idimagesimilaire=os.path.splitext(pathimagesimilaire)[0]
    classesimilaire=csvtrain.at[int(idimagesimilaire),"05_class"]
    codesimil=csvtrain.at[int(idimagesimilaire),"irma_code"]
    codesimil=codesimil.split('-')
    codesimil=codesimil[2]
    print("la classe de l'IMG req",classe ,"son code",code,"la classe de l'image similaire est",
          classesimilaire,"son code",codesimil)
    compRes=compRes+1
    if   classe==classesimilaire :
       SimImg=SimImg+1
       num =(SimImg)/compRes
       correctImages.append(num)
 if SimImg==0:
    SimImg=1
 return sum(correctImages)/SimImg

In [ ]:
def meanAveragePrecision1(maxResults):
   map = 0
   number=0
   co=0
   path ="/content/drive/MyDrive/IRMA/test/*.*"
   print(len(glob.glob(path)))
   for file in glob.glob(path):
      idimage=os.path.splitext(os.path.basename(file))[0]
      code=csvtest.at[int(idimage),"irma_code"]
      classe=csvtest.at[int(idimage),"05_class"]
      number=number+1
      """code=code.split('-')
      code=code[2]"""
      #print ("the file ", number," ",file,"classe",classe)
      if(code=="1121-120-320-700" ):
       print("the file ", number," ",file,"classe",classe," code", code)
       co=co+1
       indices_list = compute_similar_images(file, knn, device)
       indices = indices_list[0]
       ap = Averageprecision(classe,code,indices)
       print("ap",ap)
       map = map + ap
       print("map:",map)
       print("co",co)
   return map/co

In [ ]:
print("maaap:",meanAveragePrecision1(5))

1733
the file  152   /content/drive/MyDrive/IRMA/test/741634.png classe 26  code 1121-120-320-700
torch.Size([1, 3, 256, 512])
(1, 256, 8, 16)
(1, 32768)
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 26 son code 320
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 1 son code 500
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 1 son code 500
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 1 son code 500
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 1 son code 500
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 26 son code 320
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 1 son code 500
la classe de l'IMG req 26 son code 1121-120-320-700 la classe de l'image similaire est 1 son code 500
la classe de l'IMG req 26 so